### Colorectal Cancer Dataset

In [1]:
# load modules
import pandas as pd
from src import read
from src import preprocessing
from src import eda

### Data Load

In [2]:
# define data folder path and data file names

raw_data_folder = r"C:\Users\35799\Desktop\cookiecutter-analytical-project\biolizard-internship-marios\data\raw"
gene_expression_data = r"Colorectal Cancer Gene Expression Data.csv"
patient_data = r"Colorectal Cancer Patient Data.csv"

In [3]:
# load data

gene_expression_df = read.data_load(raw_data_folder, gene_expression_data)
patient_df = read.data_load(raw_data_folder, patient_data)

In [4]:
# combine data into one Pandas DataFrame

gene_expression_df_transposed = gene_expression_df.transpose()
gene_expression_df_transposed.columns = gene_expression_df_transposed.iloc[0]
gene_expression_df_transposed_index = gene_expression_df_transposed.drop("ID_REF", axis=0)
gene_expression_df_transposed_index_final = gene_expression_df_transposed_index.astype("float64")
df = pd.merge(patient_df, gene_expression_df_transposed_index_final, right_index=True, left_on='ID_REF')

In [5]:
# data info

df_id, df_cat, df_cont = read.data_info(df, threshold=20)

DATA INFO:
----------------------------------------------------------------------------------------------------


DIMENSIONS:
----------------------------------------------------------------------------------------------------
Entries: 62
Features: 1944
----------------------------------------------------------------------------------------------------


CATEGORICAL FEATURES:
----------------------------------------------------------------------------------------------------
Features     Data Type    Categories & Counts
-----------  -----------  ---------------------------------------------------
Dukes Stage  object       {'A': 16, 'B': 14, 'C': 20, 'D': 12}
Gender       object       {'Female': 14, 'Male': 48}
Location     object       {'Colon': 2, 'Left': 20, 'Rectum': 18, 'Right': 22}
DFS event    float64      {0.0: 25, 1.0: 37}
Adj_Radio    float64      {0.0: 29, 1.0: 33}
Adj_Chem     float64      {0.0: 32, 1.0: 30}
-------------------------------------------------------------------

### Data Preprocessing

In [6]:
# data split

X, y, X_train, y_train, X_test, y_test, X_val, y_val = preprocessing.data_split(df, target="Dukes Stage", method="tt", random_state=0, train_proportion=0.8, test_proportion=0.2, validation_proportion=0.25, stratify=True)

In [7]:
# treat missing values

train_df, test_df = preprocessing.treat_nan(X_train, y_train, X_test, y_test, df_id, df_cat, df_cont, target="Dukes Stage", drop_nan_rows=False, impute_cutoff=0.5, categorical_imputer="mode", continuous_imputer="median")

In [8]:
# treat duplicate values

train_df, test_df = preprocessing.treat_duplicate(train_df, test_df, keep_in="first")

In [9]:
# treat outliers

train_df, test_df = preprocessing.treat_outliers(train_df, test_df, df_id, df_cat, df_cont, target="Dukes Stage", method="svm", outlier_fraction=0.35)

The following entries are probable outliers as identified by the One-class Support Vector Machine technique (train set):


ID_REF  Gender Location  DFS event  Adj_Radio  Adj_Chem  \
59  GSM877186    Male     Left        1.0        1.0       0.0   
8   GSM877134    Male    Right        0.0        1.0       1.0   
9   GSM877135  Female   Rectum        0.0        0.0       0.0   
24  GSM877150    Male   Rectum        1.0        0.0       1.0   
35  GSM877161    Male    Right        0.0        1.0       0.0   
14  GSM877140    Male    Colon        0.0        0.0       1.0   
20  GSM877146    Male   Rectum        0.0        1.0       1.0   
39  GSM877165    Male     Left        0.0        0.0       1.0   
12  GSM877138    Male    Right        0.0        0.0       1.0   
27  GSM877153    Male   Rectum        1.0        1.0       0.0   
21  GSM877147    Male    Right        0.0        0.0       0.0   
16  GSM877142    Male    Right        1.0        0.0       1.0   
52  GSM877179    Male     Left        0.0        1.0       1.0   
45  GSM877171    Male    Right        1.0        0.0       1.0   
55  GSM877182    Male    Right        1.0        0.0       0.0   
17  GSM877143    Male    Right        0.0        1.0       0.0   

    Age (in years)  DFS (in months)     117_at  1007_s_at   1053_at    121_at  \
59            65.0             25.0   8.661629  10.427298  7.512632  7.046389   
8             54.0             80.0   4.903302  10.551750  7.869727  7.462846   
9             68.0             94.0   6.530134  10.165121  7.615570  7.476847   
24            74.0             11.0   4.908442  10.376543  8.104784  7.249637   
35            63.0             17.0   4.636176  10.064616  7.701012  7.412851   
14            63.0             68.0   5.259367  10.184694  7.983528  7.767613   
20            50.0             91.0   6.420283  10.225935  7.505152  7.113644   
39            46.0             15.0   5.774617  10.297960  7.067800  7.717166   
12            47.0             81.0   7.006106  10.033355  8.553709  7.730508   
27            61.0             14.0  10.368146  10.761687  7.237572  7.683509   
21            28.0             85.0   8.402295   9.754302  8.191826  8.588304   
16            70.0            103.0   6.004252   7.754544  6.795014  7.014675   
52            60.0             14.0   5.111097  10.084649  6.119306  7.267976   
45            73.0             24.0   4.472217  10.610713  6.754314  7.512881   
55            55.0             14.0   5.407188  10.585953  7.776103  6.898353   
17            67.0              4.0   6.468004  10.214872  7.450276  7.205786   

    1255_g_at   1294_at   1316_at   1320_at  1405_i_at   1431_at   1438_at  \
59   2.650723  5.992824  5.285014  4.047409   5.335867  3.381974  6.081336   
8    3.066619  6.982733  5.781327  3.784000   6.687646  3.349103  7.496297   
9    2.971537  7.682610  6.450320  3.681186   8.690705  3.452053  8.917090   
24   2.927889  6.674732  6.168213  3.896253   5.016757  3.288931  8.154682   
35   3.186199  7.510427  5.766115  3.829780   5.469602  3.408081  7.006718   
14   3.101163  7.057861  5.414196  3.876754   9.316734  4.758090  6.345433   
20   2.950536  7.187298  6.178597  3.688796   9.066562  3.305809  6.858216   
39   3.572938  6.694061  6.132715  3.858564   6.733462  3.327047  5.519454   
12   2.953688  7.183682  6.294097  3.985674   9.058086  3.308840  7.604331   
27   2.837354  8.969693  6.081014  4.140208  10.461856  3.418025  5.845527   
21   3.093167  6.654735  5.754420  4.062427   9.128890  3.451669  4.707356   
16   3.242701  7.833150  5.453137  3.913796  10.774562  3.419892  5.053987   
52   2.741116  8.101274  5.658214  3.979695   9.140687  3.221388  5.462382   
45   3.010147  7.829368  6.216812  4.049242   7.263656  3.409415  8.631392   
55   2.881039  7.106121  5.791258  4.019104   7.345634  3.466989  5.876684   
17   2.969755  7.172667  6.345145  3.720011   8.988367  3.476634  6.992055   

     1487_at  1494_f_at  1552256_a_at  1552257_a_at  1552258_at  1552261_at  \
59  7.960783   4.044226      7.218739      8.472297    3.486721    4.042549   
8   8.095995   4.3918

The following entries are probable outliers as identified by the One-class Support Vector Machine technique (test set):


ID_REF Gender Location  DFS event  Adj_Radio  Adj_Chem  Age (in years)  \
40  GSM877166   Male     Left        1.0        1.0       1.0            78.0   
28  GSM877154   Male   Rectum        1.0        1.0       0.0            66.0   
48  GSM877175   Male     Left        1.0        1.0       0.0            56.0   
11  GSM877137   Male    Right        0.0        1.0       0.0            68.0   
0   GSM877126   Male     Left        0.0        1.0       0.0            62.0   

    DFS (in months)    117_at  1007_s_at   1053_at    121_at  1255_g_at  \
40             17.0  4.664240  10.156231  7.726520  7.917791   3.028881   
28             13.0  5.568210  10.446765  7.385270  7.875053   2.992019   
48             12.0  4.792020  10.704085  7.718933  7.096596   2.784996   
11            106.0  5.051468  10.159113  7.384296  7.443270   2.904777   
0             108.0  6.029834   9.468893  7.187090  6.807582   3.084591   

     1294_at   1316_at   1320_at  1405_i_at   1431_at   1438_at   1487_at  \
40  7.568330  5.622042  3.855261   4.685060  3.376070  7.068375  8.258589   
28  6.835082  6.140051  3.735254   7.569299  3.560199  7.694217  7.839580   
48  7.301731  5.739711  4.002761   8.344025  3.420608  6.846713  8.837433   
11  6.961102  6.142163  3.586663   7.515161  3.586601  7.928265  8.608508   
0   7.397013  6.525426  3.848253   7.572331  3.748200  6.957488  8.057704   

    1494_f_at  1552256_a_at  1552257_a_at  1552258_at  1552261_at  1552263_at  \
40   4.337713      8.422382      8.838345    4.082848    3.270576    5.893482   
28   4.316008      8.611594      9.264851    4.225953    3.657251    6.562029   
48   4.503762      8.202261      9.512934    4.451526    3.366267    6.186895   
11   4.201323      8.808424      8.724399    4.372757    3.287648    5.928472   
0    4.311960      7.246198      8.998012    3.912720    4.206709    6.293766   

    1552264_a_at  1552266_at  1552269_at  1552271_at  1552272_a_at  \
40      6.322052    4.078892    2.852943    4.452060      3.903899   
28      7.049215    2.880847    3.088161    4.603357      4.376642   
48      6.857042    2.982286    2.956672    4.620759      4.529428   
11      6.466866    2.728882    2.999134    4.934951      4.838923   
0       6.585635    2.837637    3.024119    4.561616      4.371035   

    1552274_at  1552275_s_at  1552276_a_at  1552277_a_at  1552278_a_at  \
40    6.226961      4.582948      4.132260      7.685542      4.350138   
28    5.941120      4.549188      4.359217      7.051113      4.547219   
48    6.144479      5.342325      4.064762      6.868444      4.465585   
11    5.856147      4.425130      4.188268      6.187011      4.616938   
0     6.120874      4.361976      4.765517      7.101751      3.900317   

    1552279_a_at  1552280_at  1552281_at  1552283_s_at  1552286_at  \
40      5.542424    3.045678    7.969821      4.365200    5.714887   
28      5.546913    3.134640    7.531488      3.572419    5.711355   
48      5.234004    2.909716    8.246189      7.405030    4.962862   
11      5.714469    3.100541    8.669562      3.357916    5.997631   
0       5.035382    2.830732    6.470622      3.906776    5.081563   

    1552287_s_at  1552288_at  1552289_a_at  1552291_at  1552293_at  \
40      6.275560    3.184562      4.099147    6.782487    4.026074   
28      6.559808    3.689623      5.311167    7.256640    3.679324   
48      6.804328    3.491802      4.192631    6.924439    3.787885   
11      7.228586    2.972889      4.240298    7.267790    3.578814   
0       6.837268    3.297914      4.153212    6.358204    3.360942   

    1552295_a_at  1552296_at  1552299_at  1552301_a_at  1552302_at  \
40      5.893130    3.579156    4.307090      3.762485    3.347975   
28      6.341478    3.527600    4.500660      3.840617    4.102422   
48      6.220880    3.271156    4.791710      3.731796    3.379070   
11      6.009598    3.197234    5.069212      3.848721    2.958439   
0       5.999121    3.611584    4.041372      3.673284    4.4

In [10]:
# target feature imbalance check

preprocessing.target_balance_check(train_df, target="Dukes Stage", imbalance_fraction=0.5, graphic=False)

Target Feature Levels      Counts    Percentages
-----------------------  --------  -------------
A                               9          0.273
B                               7          0.212
C                              11          0.333
D                               6          0.182


The tagret feature levels are balanced.


In [27]:
### sampling to treat target feature imbalance

ratios = {"A": 1, "B": 1, "C": 1, "D": 1}
train_df_resampled = preprocessing.sampler(train_df, target="Dukes Stage", method="over", sampling_ratios=ratios, random_state=0, graphic=False)

Balanced target feature (oversampling):
Target Feature Levels      Counts    Percentages
-----------------------  --------  -------------
A                              11           0.25
B                              11           0.25
C                              11           0.25
D                              11           0.25


### Data Exploration

In [12]:
# data slice for examples

continuous_df = train_df.loc[:, df_cont]
slice = continuous_df.iloc[:, 2:10]

In [13]:
# correlation matrix

pearson_corr_matrix = eda.correlations(slice, type="pearson", printout="matrix")

Pearson Correlation Matrix:


,117_at,1007_s_at,1053_at,121_at,1255_g_at,1294_at,1316_at,1320_at
117_at,1.000,-0.152,0.068,-0.335,0.293,-0.130,-0.266,0.293
1007_s_at,-0.152,1.000,0.206,0.156,-0.163,-0.210,-0.056,-0.210
1053_at,0.068,0.206,1.000,-0.185,-0.139,-0.210,-0.275,0.071
121_at,-0.335,0.156,-0.185,1.000,-0.516,0.116,0.108,-0.150
1255_g_at,0.293,-0.163,-0.139,-0.516,1.000,-0.005,-0.355,0.192
1294_at,-0.130,-0.210,-0.210,0.116,-0.005,1.000,-0.174,0.199
1316_at,-0.266,-0.056,-0.275,0.108,-0.355,-0.174,1.000,-0.129
1320_at,0.293,-0.210,0.071,-0.150,0.192,0.199,-0.129,1.000


In [14]:
# correlation heatmap

spearman_corr_matrix = eda.correlations(slice, type="spearman", printout="heatmap")

#### Boxplot examples

In [15]:

eda.box_plot(train_df, x_var="", y_var=list(df.columns[9:]), group_var="", title="Gene Expression of: ", xtitle="Gene", ytitle="Gene Expression Value", widget_description="Select: ", stratify=False, group=False)

In [16]:
eda.box_plot(train_df, x_var="", y_var=list(df.columns[9:]), group_var="Gender", title="Gene Expression of: ", xtitle="Gene", ytitle="Gene Expression Value", widget_description="Select: ", stratify=False, group=True)

In [17]:
eda.box_plot(train_df, x_var="Dukes Stage", y_var=list(df.columns[9:]), group_var="", title="Gene Expression of: ", xtitle="Dukes Stage", ytitle="Gene Expression Value", widget_description="Select: ", stratify=True, group=False)

In [18]:
eda.box_plot(train_df, x_var="Dukes Stage", y_var=list(df.columns[9:]), group_var="Gender", title="Gene Expression of: ", xtitle="Dukes Stage", ytitle="Gene Expression Value", widget_description="Select: ", stratify=True, group=True)

In [19]:
eda.box_plot(train_df, x_var="Location", y_var=list(df.columns[9:]), group_var="Gender", title="Gene Expression of: ", xtitle="Location", ytitle="Gene Expression Value", widget_description="Select: ", stratify=True, group=True)

#### Histogram examples

In [20]:
eda.hist_plot(train_df, y_var=list(df.columns[9:]), group_var="", title="Gene Expression Histogram of: ", xtitle="Gene Expression Value", ytitle="Count", widget_description="Select: ", group=False)

In [21]:
eda.hist_plot(train_df, y_var=list(df.columns[9:]), group_var="Gender", title="Gene Expression Histogram of: ", xtitle="Gene Expression Value", ytitle="Count", widget_description="Select: ", group=True)

In [22]:
eda.hist_plot(train_df, y_var=list(df.columns[9:]), group_var="Dukes Stage", title="Gene Expression Histogram of: ", xtitle="Gene Expression Value", ytitle="Count", widget_description="Select: ", group=True)

In [23]:
eda.hist_plot(train_df, y_var=list(df.columns[9:]), group_var="Location", title="Gene Expression Histogram of: ", xtitle="Gene Expression Value", ytitle="Count", widget_description="Select: ", group=True)

#### Distribution plot examples

In [24]:
# data slice for examples

gene_list = list(df.columns[9:])
gene_slice = gene_list[0:10]

In [25]:
eda.dist_plot(train_df, selection=gene_slice, title="Gene Expression Distribution Plot:", xtitle="", ytitle="")

#### Dimensionality Reduction and Clustering

In [26]:
pca_df = eda.dimensionality_reduction(train_df, test_df, identifier=df_id, categorical=df_cat, continuous=list(df.columns[9:]), target="Dukes Stage", method="pca")

#### Experimental code